In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render ul li{font-size:22pt; line-height:30px;}
div.output {font-size:22pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:22pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:22pt;padding:5px;}
table.dataframe{font-size:22px;}
</style>
"""))

<b><font size="7" color="red">ch15. 데이터베이스 연동</font></b>
# 1절 SQLite 데이터 베이스 연결
- SQLite 데이터 베이스는 별도의 DBMS 없이 SQL문 이용해서 DB액세스할 수 있도록 만든 간단한 디스크 기반 DB
- C라이브러리
- SQLite는 프로토타입 생성시 사용
- 프로젝트 : 분석   ->   설계  ->  구현  -> 테스트 -> 고객에게 배포 -> 유지보수
                프로토타입(SQLite)       완제품(Oracle, MySQL, Maria, ..)
- [DB browser for SQLite](https://sqlitebrowser.org/dl/)
## 1.1 SQLite browser 설치 및 sqlite 패키지 load

In [2]:
import sqlite3
sqlite3.sqlite_version # 버전

'3.40.1'

In [3]:
import pandas as pd
pd.__version__

'1.5.3'

## 1.2 데이터 베이스 연결
- (1) 데이터베이스 연결 생성
- (2) SQL 전송 객체 생성(cursor)
- (3) SQL 전송 & 결과 받기(cursor.execute()이용)
- (4) cursor 해제 & 데이터베이스연결객체 해제

In [4]:
# (1)DB 연결 : sqlite로 연결시 DB파일이 없으면 빈 DB파일 생성하고 연결, 있으면 연결
conn = sqlite3.connect('data/ch15_example.db')
conn

In [5]:
# (2) SQL 전송 객체 생성(cursor 객체)
cursor = conn.cursor()
cursor

In [6]:
cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT, 
        AGE  INT,
        EMAIL TEXT
    )
''')

In [7]:
cursor.execute('DROP TABLE MEMBER')

In [8]:
cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT, 
        AGE  INT,
        EMAIL TEXT
    )
''')

In [9]:
cursor.execute('INSERT INTO MEMBER VALUES (\'홍길동\', 20, \'h@h.com\')')
print('수행결과 행수 :', cursor.rowcount)

수행결과 행수 : 1


In [10]:
sql = "INSERT INTO MEMBER VALUES ('김길동', 30, 'K@K.COM')"
cursor.execute(sql)
print('수행 결과 행수 :', cursor.rowcount)
cursor.execute("INSERT INTO MEMBER VALUES ('이길동', 25, 'l@l.com')")
print('수행 결과 행수 :', cursor.rowcount)

수행 결과 행수 : 1
수행 결과 행수 : 1


In [11]:
conn.commit() # (반)conn.rollback()

In [13]:
# select 전송 결과 : cursor가 가리킴
cursor.execute("SELECT * FROM MEMBER")

In [14]:
# insert, update, delete문 실행 결과 : cursor.rowcount
# select문 실행 결과 : 
        ## fetchall() : 결과를 모두 받을 때(튜플 list)
        ## fetchone() : 결과를 한행씩 받을 때(튜플)
        ## fetchmany(n) : 결과를 n행 받을 때(튜플 list)
print(cursor.fetchall())

[('홍길동', 20, 'h@h.com'), ('김길동', 30, 'K@K.COM'), ('이길동', 25, 'l@l.com')]


In [15]:
print(cursor.fetchall()) # 한번 소요된 cursor 객체는 다시 fetch될 수 없음

[]


In [16]:
# 한꺼번에 읽기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
members = cursor.fetchall()
members

[('홍길동', 20, 'h@h.com'), ('이길동', 25, 'l@l.com'), ('김길동', 30, 'K@K.COM')]

In [17]:
for member in members:
    print(member)

('홍길동', 20, 'h@h.com')
('이길동', 25, 'l@l.com')
('김길동', 30, 'K@K.COM')


In [19]:
# 하나씩 읽기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
members = []
while True:
    member = cursor.fetchone()
    if member is None:
        print('데이터 끝')
        break
    members.append(member)

데이터 끝


In [20]:
members

[('홍길동', 20, 'h@h.com'), ('이길동', 25, 'l@l.com'), ('김길동', 30, 'K@K.COM')]

In [ ]:
# 최상위 n행 읽기